In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import requests
import re

from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from time import sleep

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [4]:
# importo dal df con i risultati Booking, in modo da avere già l'elenco delle città
TA_results = pd.read_csv('booking_results.csv')
TA_results['risultati'] = np.nan

TA_results

,citta,risultati
0,Austin,NaN
1,Wrocław,NaN
2,Innsbruck,NaN
3,Villefranche-sur-Saône,NaN
4,Zakopane,NaN
...,...,...
2398,Veneto,NaN
2399,Bastia,NaN
2400,Laramie,NaN
2401,Longyearbyen,NaN


In [11]:
driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))

[WDM] - ====== WebDriver manager ======
[WDM] - Current firefox version is 99.0
[WDM] - Get LATEST geckodriver version for 99.0 firefox
[WDM] - Getting latest mozilla release info for v0.31.0
[WDM] - About to download new driver from https://github.com/mozilla/geckodriver/releases/download/v0.31.0/geckodriver-v0.31.0-linux64.tar.gz
[WDM] - Driver has been saved in cache [/home/marco/.wdm/drivers/geckodriver/linux64/v0.31.0]


In [35]:
driver.get('https://www.tripadvisor.com/Attractions')
sleep(2)
# clicco sul bottone dei cookies
driver.find_element(By.ID, 'onetrust-accept-btn-handler').click()

In [33]:
# scraping dei risultati della ricerca su TriAdvisor, per ogni città

def start_soup (link):
    response = requests.get(link, headers=headers)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup


""" Restituisce un id che Tripadvisor assegna ad ogni città. Mi servirà per accedere alla
    pagina contenente il numero di attrazioni della città. """

def search_in_text_box (city):
    driver.find_element(By.NAME, 'q').send_keys(city) # digita il nome della città nella textbox
    sleep(2)
    # controllo che siano usciti dei suggerimenti di ricerca. Se non ci sono, passo alla città successiva
    suggestions = re.findall('<a class="GzJDZ w z _S _F Wc Wh Q B- _G"', driver.page_source)
    if len(suggestions) ==1:
        return ''
    match = re.findall('-g\d+', driver.page_source)
    driver.find_element(By.NAME, 'q').clear()
    return match[0]


def scraping (df):
    for x in df['citta']:
        city_id = search_in_text_box(x)
        base_link = f"https://www.tripadvisor.com/Attractions" + city_id + "-Activities-a_allAttractions.true"

        try:
            results = start_soup(base_link)
            number_of_attractions = results.find('div', class_='Ci').text.split()[-1]
            # inserisco il numero nel dataframe, in corrispondenza della città
            df['risultati'][df['citta'] == x] = number_of_attractions
        except AttributeError:
            try:
                number_of_attractions = results.find('div', class_='aTSjG').text.split()[0]
                df['risultati'][df['citta'] == x] = number_of_attractions
            except:
                df['risultati'][df['citta'] == x] = np.nan
        except Exception as e:
            df['risultati'][df['citta'] == x] = np.nan
    
    return df
       
    
# senza l'header mi restituisce la pagina di accettazione cookie, quindi non riesco a estrarre il numero di risultati
headers = {
    'User-agent':  'Mozilla/5.0 (X11; Linux x86_64; rv:99.0) Gecko/20100101 Firefox/99.0' }

TA_results = scraping(TA_results)

In [36]:
TA_results.to_csv('tripadvisor_results.csv', index=False)

In [35]:
# vedo quanti valori non sono riuscito ad estrarre
print(len(TA_results[TA_results['risultati'].isna()]))

28

In [ ]:
"""Altre osservazioni sui risultati:

    1. Scorrendo la lista, noto che ci sono 6 luoghi che hanno esattamente 259.406 attrazioni,
    numero troppo grande. Andando a cercare manualmente questi luoghi, vedo che la ricerca non
    restituisce i suggerimenti, da cui poi normalmente estrarrei l'id della città.
    Correggo questo bug nella funzione search_in_text_box, andando a controllare se ci sono
    suggerimenti di ricerca.
    
    2. alcuni luoghi hanno un numero elevato di attrazioni, perchè non sono città
       o province, bensì sono degli Stati: sono poco più di 20
"""